In [ ]:
# POST /combine

import pandas as pd

# idA = "0EF6564F-535E-6660-BB4B-B8BEC4CC9236"
# idB = "0EF07E06-6C61-7120-A431-CF6973583BC3"
# op = "+"
# workbook_id = "0EF5FE4C-67A2-62D0-9752-BB9667D7E30B"
# worksheet_id = "0EF5FE4C-6B7C-E810-9710-6EBDA5E189C9"

# Get params from request body
idA = REQUEST['body']['idA']
idB = REQUEST['body']['idB']
op = REQUEST['body']['op']
workbook_id = REQUEST['body']['workbookId']
worksheet_id = REQUEST['body']['worksheetId']

# Step 2: Create the formula to add the two signals
metadata = pd.DataFrame([{
    'Name': 'Summed Signal',
    'Formula': f"$signalA.setUnits('') {op} $signalB.setUnits('')",
    'Formula Parameters': {
        'signalA': idA,
        'signalB': idB
    },
    'Type': 'Formula'
}])

# Step 3: Push the formula to Seeq
result = spy.push(
    workbook=workbook_id,
    metadata=metadata
)

# Extract the ID of the new signal
new_signal_id = result['ID'].values[0]

# Get the current worksheet
workbook = spy.workbooks.pull(workbook_id)[0]
worksheet = next((ws for ws in workbook.worksheets if ws.id == worksheet_id), None)

# Add the new signal to the worksheet's details pane
# worksheet.display_items.append({
#     'ID': new_signal_id,
#     'Type': 'Signal'
# })
worksheet.display_items = pd.concat([worksheet.display_items, result]).reset_index()

# Push the updated worksheet back to Seeq
spy.workbooks.push(workbook)